<a href="https://colab.research.google.com/github/PrasoonPratham/tensorflow-etl-benchmark/blob/main/ETL_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow_datasets as tdfs
import tensorflow_addons as tfa

In [2]:
data = tdfs.load("malaria", split='train', as_supervised=True)

2022-06-22 00:55:29.738636: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:34<00:00, 34.52s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:39<00:00, 39.39s/ url]
                                                                        

Dataset malaria downloaded and prepared to ~/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-06-22 00:56:17.265344: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-22 00:56:17.265505: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Importing the inception V3 model
from tensorflow.keras.applications.inception_v3 import InceptionV3
# Downloading the weights
import urllib

weights = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

weights_file = 'inception_v3.h5'
urllib.request.urlretrieve(weights, weights_file)
# Loading the weights
pretrained_model = InceptionV3(
    input_shape=(100, 100, 3),
    include_top=False,
    weights=None
)

pretrained_model.load_weights(weights_file)
pretrained_model.summary()
for layer in pretrained_model.layers:
    layer.trainable = False

last_layer = pretrained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
from tensorflow.keras.layers import Flatten, Dense

x = Flatten()(last_output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)
from tensorflow.keras import Model

model = Model(pretrained_model.input, x)
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 49, 49, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 49, 49, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [8]:
def augmentation(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (100, 100))
    image = image / 255
    image = tf.image.random_flip_left_right(image)
    image = tfa.image.rotate(image, 40, interpolation='NEAREST')
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label

In [9]:
train = data.map(augmentation)
train_batches = train.shuffle(100).batch(32)

In [10]:
# Training the model
history = model.fit(
    train_batches,
    epochs=15
)

Epoch 1/15


2022-06-22 01:03:59.801689: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-22 01:03:59.801978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


862/862 [==============================] - 35s 39ms/step - loss: 0.2523 - accuracy: 0.8995
Epoch 2/15
862/862 [==============================] - 33s 38ms/step - loss: 0.2009 - accuracy: 0.9245
Epoch 3/15
862/862 [==============================] - 33s 38ms/step - loss: 0.1837 - accuracy: 0.9331
Epoch 4/15
862/862 [==============================] - 33s 38ms/step - loss: 0.1774 - accuracy: 0.9361
Epoch 5/15
862/862 [==============================] - 33s 38ms/step - loss: 0.1711 - accuracy: 0.9389
Epoch 6/15
862/862 [==============================] - 33s 38ms/step - loss: 0.1633 - accuracy: 0.9416
Epoch 7/15
862/862 [==============================] - 33s 38ms/step - loss: 0.1568 - accuracy: 0.9443
Epoch 8/15
862/862 [==============================] - 33s 38ms/step - loss: 0.1519 - accuracy: 0.9467
Epoch 9/15
862/862 [==============================] - 33s 38ms/step - loss: 0.1488 - accuracy: 0.9481
Epoch 10/15
862/862 [==============================] - 33s 38ms/step - loss: 0.1400 - accurac

In [11]:
file_pattern = f'~/tensorflow_datasets/malaria/1.0.0/malaria-train.tfrecord*'
file = tf.data.Dataset.list_files(file_pattern)

In [12]:
train_dataset = file.interleave(
    tf.data.TFRecordDataset,
    cycle_length=4,
    num_parallel_calls=tf.data.experimental.AUTOTUNE
    )

In [13]:
def augmentationV2(serialized_data):
    description ={
        'image': tf.io.FixedLenFeature((), tf.string),
        'label': tf.io.FixedLenFeature((), tf.int64, -1)
    }
    example = tf.io.parse_single_example(serialized_data, description)
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (100, 100))
    image = image / 255
    image = tf.image.random_flip_left_right(image)
    image = tfa.image.rotate(image, 40, interpolation='NEAREST')
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, example['label']

In [14]:
import multiprocessing
cores = multiprocessing.cpu_count()

print(cores)

8


In [15]:
train_dataset = train_dataset.map(augmentationV2, num_parallel_calls=cores)
train_dataset = train_dataset.cache()

In [16]:
train_dataset = train_dataset.shuffle(100).batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [17]:
# Training the model
history = model.fit(
    train_dataset,
    epochs=15
)

Epoch 1/15
862/862 [==============================] - 28s 32ms/step - loss: 0.1143 - accuracy: 0.9621
Epoch 2/15
862/862 [==============================] - 26s 30ms/step - loss: 0.0997 - accuracy: 0.9680
Epoch 3/15
862/862 [==============================] - 26s 30ms/step - loss: 0.0874 - accuracy: 0.9720
Epoch 4/15
862/862 [==============================] - 26s 30ms/step - loss: 0.0778 - accuracy: 0.9761
Epoch 5/15
862/862 [==============================] - 27s 31ms/step - loss: 0.0678 - accuracy: 0.9794
Epoch 6/15
862/862 [==============================] - 27s 31ms/step - loss: 0.0593 - accuracy: 0.9821
Epoch 7/15
862/862 [==============================] - 27s 31ms/step - loss: 0.0521 - accuracy: 0.9855
Epoch 8/15
862/862 [==============================] - 27s 31ms/step - loss: 0.0451 - accuracy: 0.9874
Epoch 9/15
862/862 [==============================] - 26s 30ms/step - loss: 0.0388 - accuracy: 0.9891
Epoch 10/15
862/862 [==============================] - 27s 31ms/step - loss: 0.033